In [34]:
pip install boto3 pandas openpyxl xlrd gspread oauth2client

Note: you may need to restart the kernel to use updated packages.


In [35]:
import pandas as pd
import boto3
import os
from dotenv import load_dotenv
import gspread
from oauth2client.service_account import ServiceAccountCredentials

In [36]:
load_dotenv()

True

In [37]:
access_key = os.getenv("AWS_ACCESS_KEY_ID")
secret_key = os.getenv("AWS_SECRET_ACCESS_KEY")
BUCKET_NAME = os.getenv("BUCKET_NAME")

In [38]:
s3 = boto3.client(
    "s3",
    aws_access_key_id = access_key ,
    aws_secret_access_key = secret_key 
)


In [39]:
def download_xl_file(bucket_name , s3_key , local_file) :
    try :
        s3.download_file(bucket_name , s3_key , local_file)
        print ("download success")
    except Exception as e :
        print (f"error : '{e}'")

In [40]:
def upload_csv_file(bucket_name , s3_key , local_file) :
    try :
        s3.upload_file(local_file , bucket_name , s3_key)
        print ("upload success")
    except Exception as e :
        print (f"error : '{e}'")

In [41]:
s3_key = "sondagepublic/sondage.xlsx"
local_file = 'sondage.xlsx'
download_xl_file(BUCKET_NAME , s3_key , local_file)

download success


In [42]:
data = pd.read_excel(local_file , engine = "openpyxl")
print(data)

        product  quantity        date  price_before  price_after  \
0       chemise         5       45750          1200          100   
1  Paquet de 12        10       45764            60          100   
2            P1        10       45607            10            8   
3       montres         4       45762            50           35   
4  paquet de 12        10  2025-04-19            20           99   
5     nettoyant        10  2025-04-04          5000         6000   

       prom_type gender  age  income  
0  offre grouper  homme   21      45  
1      réduction  homme   22   10000  
2      réduction  homme   30      10  
3  offre grouper  femme   20     500  
4    bon d'achat  homme   16     100  
5      réduction  homme   25    5000  


In [43]:
def delete_empty_row_col(df) :
    df.dropna(how = 'all' , inplace = True)
    df.dropna(axis=1 , how = 'all' , inplace = True)


In [44]:
def rename_col(df) :
    df.columns = df.columns.str.strip().str.lower().str.replace(' ','_')

In [45]:
def delete_duplicate(df) :
    df.drop_duplicates(inplace=True)

In [46]:
def replace_product(df) :
    df["product"] = df["product"].fillna("unknown")


In [47]:
def replace_qty(df) :
    df["quantity"] = df["quantity"].fillna(1) 

In [48]:
def ajust_prices(df) :
    df.dropna(subset=["price_before"] , inplace=True)
    df.dropna(subset=["price_after"] , inplace = True)
    before = df["price_before"]
    after = df["price_after"]
    mask = df["price_after"] > df["price_before"]
    temp = df.loc[mask, "price_before"].copy()
    df.loc[mask, "price_before"] = df.loc[mask, "price_after"]
    df.loc[mask, "price_after"] = temp

In [49]:
delete_empty_row_col(data)
rename_col(data) 
delete_duplicate(data)
replace_product(data)
replace_qty(data)
ajust_prices(data)

In [54]:
data.to_csv('clean_data.csv' , index=False)

In [51]:
upload_csv_file(BUCKET_NAME , "sondagepublic/clean_sondage.csv" , "clean_data.csv")

upload success


In [52]:
print(data)

        product  quantity        date  price_before  price_after  \
0       chemise         5       45750          1200          100   
1  Paquet de 12        10       45764           100           60   
2            P1        10       45607            10            8   
3       montres         4       45762            50           35   
4  paquet de 12        10  2025-04-19            99           20   
5     nettoyant        10  2025-04-04          6000         5000   

       prom_type gender  age  income  
0  offre grouper  homme   21      45  
1      réduction  homme   22   10000  
2      réduction  homme   30      10  
3  offre grouper  femme   20     500  
4    bon d'achat  homme   16     100  
5      réduction  homme   25    5000  


In [55]:
print(data)

        product  quantity        date  price_before  price_after  \
0       chemise         5       45750          1200          100   
1  Paquet de 12        10       45764           100           60   
2            P1        10       45607            10            8   
3       montres         4       45762            50           35   
4  paquet de 12        10  2025-04-19            99           20   
5     nettoyant        10  2025-04-04          6000         5000   

       prom_type gender  age  income  
0  offre grouper  homme   21      45  
1      réduction  homme   22   10000  
2      réduction  homme   30      10  
3  offre grouper  femme   20     500  
4    bon d'achat  homme   16     100  
5      réduction  homme   25    5000  


In [58]:
scope = ["https://spreadsheets.google.com/feeds", "https://www.googleapis.com/auth/drive"]
creds = ServiceAccountCredentials.from_json_keyfile_name('sheet_secret.json', scope)
client = gspread.authorize(creds)
spreadsheet = client.open("analyse") 
worksheet = spreadsheet.sheet1
for row in data.values.tolist():
    worksheet.append_row(row)

print("Les données ont été téléchargées dans Google Sheets.")

Les données ont été téléchargées dans Google Sheets.
